# Importing libraries

In [ ]:
import os, glob, json, csv, subprocess, sys, re, operator
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import itertools

# Defining repository

In [ ]:
userhome = os.path.expanduser("~")
repository = userhome + r"/different-diff/dataset/hbase/"

In [ ]:
analyze_dir = userhome + r"/different-diff/analyze/analyze_hbase/"

# Defining function to execute git command

In [ ]:
def execute_command(cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Finding the history of files

In [ ]:
colnames = ['bug_id','commit_id','parent_id','filepath','filename','number_of_buggyline']

# Listing files from data

In [ ]:
#Show the csv file
diffbugs_hist = pd.read_csv(analyze_dir + 'file-diff/number_of_files/histogram_total_bugline.csv')
diffbugs_hist = diffbugs_hist[colnames]
diffbugs_min = pd.read_csv(analyze_dir + 'file-diff/number_of_files/minimal_total_bugline.csv')
diffbugs_min = diffbugs_min[colnames]
diffbugs_myers = pd.read_csv(analyze_dir + 'file-diff/number_of_files/myers_total_bugline.csv')
diffbugs_myers = diffbugs_myers[colnames]
diffbugs_pat = pd.read_csv(analyze_dir + 'file-diff/number_of_files/patience_total_bugline.csv')
diffbugs_pat = diffbugs_pat[colnames]

In [ ]:
diffbugs_hist.iloc[571][3]

In [ ]:
diffbugs_hist

# Applying "git blame" for files

In [ ]:
pattern1 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+(?P<filename>[^\s]+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>.*)')
pattern2 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>.*)')

In [ ]:
header1 = ['commit_id', 'filename', 'committer', 'date', 'time', 'line_number', 'code']
header2 = ['commit_id', 'committer', 'date', 'time', 'line_number', 'code']

In [ ]:
blame_dirs = analyze_dir + 'step_3/blame_files/'

# blame file for histogram

In [ ]:
filename_hist = open(analyze_dir + 'file-diff/filename/filename_histogram.txt')
filename_hist = filename_hist.read().split('\n')
filename_hist.sort()
filename_hist[571]

In [ ]:
len(filename_hist)

In [ ]:
diffbugs_hist.iloc[571][3]

In [ ]:
errorfile_hist = []
for cc, fn in enumerate(filename_hist):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(filename_hist)))
    sys.stdout.flush()
    
    parent = diffbugs_hist.iloc[cc][2]
    file = diffbugs_hist.iloc[cc][3]
    
    blametxt = analyze_dir + 'step_3/blame_files/blame_histogram/txt/blame_' + filename_hist[cc] + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + file + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + 'step_3/blame_files/blame_histogram/csv/blame_' + filename_hist[cc] + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [parent, filename_hist[cc], file]
                errorfile_hist.append(tmp)
                print (' : ' + filename_hist[cc] + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

In [ ]:
with open(analyze_dir + 'step_3/blame_files/blame_histogram/error_files_histogram.csv','w') as errorfile:
    header = ['parent_id', 'filename', 'filepath']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_hist:
        writers.writerow(file)

# blame file for minimal

In [ ]:
filename_min = open(analyze_dir + 'file-diff/filename/filename_minimal.txt')
filename_min = filename_min.read().split('\n')
filename_min.sort()
filename_min[765]

In [ ]:
diffbugs_min.iloc[765][3]

In [ ]:
errorfile_min = []
for cc, fn in enumerate(filename_min):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(filename_min)))
    sys.stdout.flush()
    
    parent = diffbugs_min.iloc[cc][2]
    file = diffbugs_min.iloc[cc][3]
    
    blametxt = analyze_dir + 'step_3/blame_files/blame_minimal/txt/blame_' + filename_min[cc] + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + file + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + 'step_3/blame_files/blame_minimal/csv/blame_' + filename_min[cc] + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [parent, filename_min[cc], file]
                errorfile_min.append(tmp)
                print (' : ' + filename_min[cc] + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

In [ ]:
with open(analyze_dir + 'step_3/blame_files/blame_minimal/error_files_minimal.csv','w') as errorfile:
    header = ['parent_id', 'filename', 'filepath']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_min:
        writers.writerow(file)

# blame for myers

In [ ]:
filename_myers = open(analyze_dir + 'file-diff/filename/filename_myers.txt')
filename_myers = filename_myers.read().split('\n')
filename_myers.sort()
filename_myers[345]

In [ ]:
diffbugs_myers.iloc[345][3]

In [ ]:
errorfile_myers = []
for cc, fn in enumerate(filename_myers):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(filename_myers)))
    sys.stdout.flush()
    
    parent = diffbugs_myers.iloc[cc][2]
    file = diffbugs_myers.iloc[cc][3]
    
    blametxt = analyze_dir + 'step_3/blame_files/blame_myers/txt/blame_' + filename_myers[cc] + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + file + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + 'step_3/blame_files/blame_myers/csv/blame_' + filename_myers[cc] + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [parent, filename_myers[cc], file]
                errorfile_myers.append(tmp)
                print (' : ' + filename_myers[cc] + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

In [ ]:
with open(analyze_dir + 'step_3/blame_files/blame_myers/error_files_myers.csv','w') as errorfile:
    header = ['parent_id', 'filename', 'filepath']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_myers:
        writers.writerow(file)

# blame for patience

In [ ]:
filename_pat = open(analyze_dir + 'file-diff/filename/filename_patience.txt')
filename_pat = filename_pat.read().split('\n')
filename_pat.sort()
filename_pat[1354]

In [ ]:
diffbugs_pat.iloc[1354][3]

In [ ]:
errorfile_pat = []
for cc, fn in enumerate(filename_pat):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(filename_pat)))
    sys.stdout.flush()
    
    parent = diffbugs_pat.iloc[cc][2]
    file = diffbugs_pat.iloc[cc][3]
    
    blametxt = analyze_dir + 'step_3/blame_files/blame_patience/txt/blame_' + filename_pat[cc] + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + file + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt,'r')
    c = b.read().split('\n')

    name = analyze_dir + 'step_3/blame_files/blame_patience/csv/blame_' + filename_pat[cc] + '_' + str(cc+1) + '.csv'
    with open(name, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            try:
                pattern1.match(c[0]).groups()
                writer.writerow(header1)
                for line in c:
                    writer.writerow(pattern1.match(line).groups())
            except:
                pattern2.match(c[0]).groups()
                writer.writerow(header2)
                for line in c:
                    writer.writerow(pattern2.match(line).groups())
        except:
            if os.stat(blametxt).st_size == 0:
                writer.writerow(header1)
                tmp = [parent, filename_pat[cc], file]
                errorfile_pat.append(tmp)
                print (' : ' + filename_pat[cc] + ' --> file not found')
            pass

#exit from checkout
excheck = "git checkout -f master"
execute_command(excheck, repository)
print ("\n'git blame' implementation is complete")

In [ ]:
with open(analyze_dir + 'step_3/blame_files/blame_patience/error_files_patience.csv','w') as errorfile:
    header = ['parent_id', 'filename', 'filepath']
    writers = csv.writer(errorfile)
    writers.writerow(header)
    for file in errorfile_pat:
        writers.writerow(file)

# Delete the txt files from git blame process (unnecessary for next steps)

In [ ]:
folder = ['blame_histogram','blame_minimal','blame_myers','blame_patience']
for z, fold in enumerate(folder):
    c = 1
    for file in glob.iglob(analyze_dir + 'step_3/blame_files/' + fold + '/txt/*', recursive=True):
        sys.stdout.write('\r%i ' %(z+1) + 'Deleting file: %i ' %c)
        sys.stdout.flush()
        c += 1
        try:
            os.remove(file)
        except FileNotFoundError:
            print ("\nError: %s" % (file))
print ('\n')
print ('Deleting files are complete')